<h1> One file to gather all raw data and convert to cleaned, useable, csv files </h1>

<h1> 1) Weightlifting Dataset </h1>

Based on data from [Summer and Winter Olympics, 1896-2022](https://www.kaggle.com/datasets/piterfm/olympic-games-medals-19862018/).
Handled by Nicholas Stach.

<h2> Description </h2>

This dataset is on Olympic games for both the Summer and Winter Olympics, from 1896 to 2022. The data contains all events of each Olympic game and the medal results, i.e., which countries won a bronze, silver, or gold medal for each event. This data was harvested from the official [Olympics website](https://olympics.com/). For each Olympic games in the dataset, it is broken down by event, categories within the event (by gender), and the medal won by the athlete or team, and their associated country. For this project, the dataset will be trimmed down to focus exclusively on the weightlifting events from 1896-2020 (the most recent summer games in the dataset).

### Data Quirks:

1) Weight classes have changed numerous times over the course of Olympic game history, and are thus not uniform!
2) There was no weightlifting event for the 2022 Beijing Games. Weightlifting is a summer sport and the most recent game recorded was a winter game.

In [39]:
# Import modules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# Import data & clean
# From: https://www.kaggle.com/datasets/piterfm/olympic-games-medals-19862018/
filePath = '../Raw Data/Summer-Winter Olympics/'
fileName = 'olympic_medals.csv'
colHeads = ["discipline_title", "oly_game", "event_title", "event_gender", "medal_type", "participant_type", 
            "particpant_title", "athlete_url", "athlete_full_name", "country_name", "country_code_2_letter", "country_code"]        # column names
allSports = pd.read_csv(filePath+fileName, skiprows=1, names=colHeads)                                                              # read in all data (every sport)

# Focus exclusively on weightlifting
weightDF = allSports[allSports['discipline_title'] == 'Weightlifting']                                                              # all weightlifting medalists, by game, 1896-2022

# Trim data of labels that are redundant for our purposes
weightDF = weightDF.drop(columns=["discipline_title", "participant_type", "particpant_title", "athlete_url", 
                                  "country_code_2_letter"], axis=1)

# Add explicit year column (to help in data analysis later)
years = pd.Series(weightDF["oly_game"].str[-4:], name='year', dtype='int')                                                          # get year for each row (last four characters of 'oly_game')
weightDF = pd.concat([weightDF, years], axis=1)                                                                                     # concatenate, column-wise
# print("Cleaned data: ")
# display(weightDF)

# Create file for cleaned data
weightDF.to_csv("weightlifting medals.csv", index=False)

df1 = pd.read_csv("./weightlifting medals.csv")
print("New file for cleaned data: ")
display(df1)

New file for cleaned data: 


,oly_game,event_title,event_gender,medal_type,athlete_full_name,country_name,country_code,year
0,tokyo-2020,Men's 61kg,Men,SILVER,Eko Yuli IRAWAN,Indonesia,INA,2020
1,tokyo-2020,Men's 61kg,Men,GOLD,Fabin LI,People's Republic of China,CHN,2020
2,tokyo-2020,Men's 61kg,Men,BRONZE,Igor SON,Kazakhstan,KAZ,2020
3,tokyo-2020,Women's 55kg,Women,SILVER,Qiuyun LIAO,People's Republic of China,CHN,2020
4,tokyo-2020,Women's 55kg,Women,BRONZE,Zulfiya CHINSHANLO,Kazakhstan,KAZ,2020
...,...,...,...,...,...,...,...,...
674,athens-1896,heavyweight - one hand lift men,Men,SILVER,Viggo JENSEN,Denmark,DEN,1896
675,athens-1896,heavyweight - one hand lift men,Men,BRONZE,Alexandros Nikolopoulos,Greece,GRE,1896
676,athens-1896,heavyweight - two hand lift men,Men,GOLD,Viggo JENSEN,Denmark,DEN,1896
677,athens-1896,heavyweight - two hand lift men,Men,SILVER,Launceston ELLIOT,Great Britain,GBR,1896


<h1> 2) Cost of Healthy Diet Dataset </h1>
Based on this [dataset](https://databank.worldbank.org/source/food-prices-for-nutrition).
Handled by Thibault Eelen.


<h2> Description </h2>
Food Prices for Nutrition provides indicators on the cost and affordability of healthy diets in each country, showing the population’s physical and economic access to sufficient quantities of locally available items for an active and healthy life over 5 years (2017-2021). It also provides indicators on the cost and affordability of an energy-sufficient diet and a nutrient-adequate diet. It also has a column with each country's population each year. 


### Data Quirks:

1) Not every country has values in every column, for every year. This will require some handling of NaNs and adjustment of averages.
2) Does not share all country names and country codes with Weightlifting data, will need extra care when comparing by country across datasets.

### NOTE:
    
1) 'millions_healthy_diet' -> millions who cannot afford a healthy diet at 52% of income
2) 'millions_nutrient' -> millions who cannot afford nutrient adequacy at 52% of income
3) 'millions_calories' -> millons who cannot afford sufficient calories at 52% of income
4) 'percent_healthy_diet' -> percent of pop who cannot afford healthy diet
5) 'percent_nutrient' -> percent of pop who cannot afford nutrient adequacy
6) 'percent_calories' -> percent of pop who cannot afford sufficient calories
7) 'affordability_energy' -> affordability of energy sufficient diet (ratio of cost to food expenditures)
8) 'affordability_nutrient' -> affordability of nutrient adequate diet (ratio of cost to food expenditures)
9) 'affodability_healthy_diet' -> affordability of healthy diet (ratio of cost to food expenditures)


In [40]:
# Import data & clean
# From: https://databank.worldbank.org/source/food-prices-for-nutrition
path2file = '../Raw Data/Healthy Diet and Populations/'
fn = 'cost of healthy diet.csv'
colHeads = ["Classification Name", "Classification Code", "country_name", "country_code", "year", "Time Code", "population",
            "millions_healthy_diet", "millions_nutrient", "millions_calories", "percent_healthy_diet", "percent_nutrient",
            "percent_calories", "affordability_energy", "affordability_nutrient", "affordability_healthy_diet" ]
DF = pd.read_csv(path2file+fn, skiprows=1, names=colHeads, encoding='iso-8859-1') 


DFDiet = DF.drop(columns=["Classification Name", "Classification Code", "Time Code", ])  # remove unneeded columns
DFDiet = DFDiet.drop([930, 931, 932, 933, 934])                            # delete update and author info at bottom of file
DFDiet.replace('..', np.nan, inplace=True)                                 # turn invalid data ('..') into NaN for easy exclusion during analysis

columnTypes = {'country_name': str,
               'country_code': str,
               'year': float, 
               'population': float,
               'millions_healthy_diet': float,
                'millions_nutrient': float,
                'millions_calories': float,
                'percent_healthy_diet': float,
                'percent_nutrient': float,
                'percent_calories': float,
                'affordability_energy': float,
                'affordability_nutrient': float,
                'affordability_healthy_diet': float}                
DFDiet = DFDiet.astype(columnTypes)                                     # convert every column to expected dtype

# print("Cleaned data: ")
# display(DFDiet)

# Create file for cleaned data
DFDiet.to_csv("healthy diet.csv", index=False)
df2 = pd.read_csv("./healthy diet.csv")
print("New file for cleaned data: ")
display(df2)

New file for cleaned data: 


,country_name,country_code,year,population,millions_healthy_diet,millions_nutrient,millions_calories,percent_healthy_diet,percent_nutrient,percent_calories,affordability_energy,affordability_nutrient,affordability_healthy_diet
0,Albania,ALB,2017.0,2873457.0,0.9,0.3,0.0,31.3,8.9,0.0,0.078,0.266,0.425
1,Albania,ALB,2018.0,2866376.0,0.7,NaN,NaN,23.0,NaN,NaN,NaN,NaN,NaN
2,Albania,ALB,2019.0,2854191.0,0.6,NaN,NaN,22.2,NaN,NaN,NaN,NaN,NaN
3,Albania,ALB,2020.0,2837849.0,0.6,NaN,NaN,19.9,NaN,NaN,NaN,NaN,NaN
4,Albania,ALB,2021.0,2811666.0,0.4,NaN,NaN,15.7,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
925,Zimbabwe,ZWE,2017.0,14751101.0,10.0,1.6,0.0,67.8,10.6,0.0,0.106,0.314,0.973
926,Zimbabwe,ZWE,2018.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
927,Zimbabwe,ZWE,2019.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
928,Zimbabwe,ZWE,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<h1> 3) Title </h1>



<h2> Description </h2>



### Data Quirks:

1) 

